In [1]:
#import dependecies
import pandas as pd
from datetime import datetime
from config import weather_api_key
from config import g_key
from citipy import citipy
import numpy as np
import requests


In [2]:
#create random latitudes and longitudes
lats = np.random.uniform(low=-90.000,high= 90.000, size=2000)
lngs = np.random.uniform(low=-180.000,high=180.000, size=2000)

In [3]:
#list comprehension used with citipy module to iterate through random lats & lngs to create a list of tuples containing the city name and the country name
city_names_country_code=[(citipy.nearest_city(i,v).city_name,citipy.nearest_city(i,v).country_code) for i,v in zip(lats,lngs)]

In [4]:
# for i,v in city_names_country_code:print(i,v)

In [ ]:
#query API (start query)
print("Beginning Data Retrieval")
print("-"*30)

#accumulator to build Pandas DataFrame
world_data=[]

#index for record count / percentage
index=0

#iterate through list of tuples containing city_names (i) and country_code (v) 
for i,v in city_names_country_code:
    
    #keep track of number of loops with index
    index+=1
    
    #create a dynamic url that queries based on city_name (i)
    url="https://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID="+weather_api_key+"&q="+i
    
    #use the request library to request the data for the current url
    response=requests.get(url)
    
    #convert the reponse into a json object/dictionary
    json_dict=response.json()
    
    #utilize try/except to check the currect json object for our wanted keys and values, saved to a variable without throwing an error if key does not exist in object
    try:
        city_lat = json_dict["coord"]["lat"]
        city_lng = json_dict["coord"]["lon"]
        city_max_temp = json_dict["main"]["temp_max"]
        city_humidity = json_dict["main"]["humidity"]
        city_clouds = json_dict["clouds"]["all"]
        city_wind = json_dict["wind"]["speed"]
        city_country = json_dict["sys"]["country"]
        
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(json_dict["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information into world_data list.
        world_data.append({"City": i,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})
        
        #After the data is succesfully collected print the following message: city, country, iteration (index), and percentage (index/len(city_names_country_code))
        print(f'city:    {i.title()}\ncountry: {v.upper()}\n\t{response} {index} of {len(city_names_country_code)} | {"{:.2f}".format((index/len(city_names_country_code)*100))}% completed\n')
    
    #KeyError would reflect that the json object does not contain the information we are looking for. we catch and skip to the next object
    except KeyError:
        print(f"\n\nWeather data for City {i.title()} in Country {v.upper()} not found. Skipping...\n\n")
        pass
        
print("-"*30)
print("Data Retrieval Complete      ")
print("-"*30)


Beginning Data Retrieval
------------------------------
city:    Bluff
country: NZ
	<Response [200]> 1 of 2000 | 0.05% completed

city:    Nikolskoye
country: RU
	<Response [200]> 2 of 2000 | 0.10% completed

city:    Busselton
country: AU
	<Response [200]> 3 of 2000 | 0.15% completed

city:    Upernavik
country: GL
	<Response [200]> 4 of 2000 | 0.20% completed

city:    Nacala
country: MZ
	<Response [200]> 5 of 2000 | 0.25% completed

city:    Nikolskoye
country: RU
	<Response [200]> 6 of 2000 | 0.30% completed



Weather data for City Amderma in Country RU not found. Skipping...


city:    Mahebourg
country: MU
	<Response [200]> 8 of 2000 | 0.40% completed

city:    Itarema
country: BR
	<Response [200]> 9 of 2000 | 0.45% completed

city:    Rikitea
country: PF
	<Response [200]> 10 of 2000 | 0.50% completed

city:    Victoria
country: SC
	<Response [200]> 11 of 2000 | 0.55% completed

city:    Puerto Ayora
country: EC
	<Response [200]> 12 of 2000 | 0.60% completed

city:    Yellowknif

city:    Riviere-Au-Renard
country: CA
	<Response [200]> 106 of 2000 | 5.30% completed

city:    Progreso
country: MX
	<Response [200]> 107 of 2000 | 5.35% completed

city:    Ushuaia
country: AR
	<Response [200]> 108 of 2000 | 5.40% completed

city:    Hammerfest
country: NO
	<Response [200]> 109 of 2000 | 5.45% completed

city:    Andenes
country: NO
	<Response [200]> 110 of 2000 | 5.50% completed

city:    Saskylakh
country: RU
	<Response [200]> 111 of 2000 | 5.55% completed



Weather data for City Saleaula in Country WS not found. Skipping...


city:    Vaini
country: TO
	<Response [200]> 113 of 2000 | 5.65% completed

city:    Albany
country: AU
	<Response [200]> 114 of 2000 | 5.70% completed

city:    Sao Filipe
country: CV
	<Response [200]> 115 of 2000 | 5.75% completed

city:    Punta Arenas
country: CL
	<Response [200]> 116 of 2000 | 5.80% completed

city:    Mataura
country: PF
	<Response [200]> 117 of 2000 | 5.85% completed

city:    Nikolskoye
country: RU
	<Response [200]>

city:    Qaanaaq
country: GL
	<Response [200]> 210 of 2000 | 10.50% completed

city:    Clonakilty
country: IE
	<Response [200]> 211 of 2000 | 10.55% completed

city:    Lloydminster
country: CA
	<Response [200]> 212 of 2000 | 10.60% completed

city:    Votkinsk
country: RU
	<Response [200]> 213 of 2000 | 10.65% completed

city:    Biarritz
country: FR
	<Response [200]> 214 of 2000 | 10.70% completed



Weather data for City Yialos in Country GR not found. Skipping...


city:    Caravelas
country: BR
	<Response [200]> 216 of 2000 | 10.80% completed



Weather data for City Taolanaro in Country MG not found. Skipping...


city:    Yellowknife
country: CA
	<Response [200]> 218 of 2000 | 10.90% completed

city:    Codrington
country: AG
	<Response [200]> 219 of 2000 | 10.95% completed

city:    Punta Arenas
country: CL
	<Response [200]> 220 of 2000 | 11.00% completed

city:    Narsaq
country: GL
	<Response [200]> 221 of 2000 | 11.05% completed

city:    Jamestown
country: SH
	<Response [2

city:    Avarua
country: CK
	<Response [200]> 313 of 2000 | 15.65% completed

city:    Kapaa
country: US
	<Response [200]> 314 of 2000 | 15.70% completed

city:    Adrar
country: DZ
	<Response [200]> 315 of 2000 | 15.75% completed

city:    Ponta Do Sol
country: CV
	<Response [200]> 316 of 2000 | 15.80% completed

city:    Mataura
country: PF
	<Response [200]> 317 of 2000 | 15.85% completed

city:    Yellowknife
country: CA
	<Response [200]> 318 of 2000 | 15.90% completed

city:    Rimoc
country: HU
	<Response [200]> 319 of 2000 | 15.95% completed

city:    Rikitea
country: PF
	<Response [200]> 320 of 2000 | 16.00% completed

city:    Kaitangata
country: NZ
	<Response [200]> 321 of 2000 | 16.05% completed

city:    Glens Falls
country: US
	<Response [200]> 322 of 2000 | 16.10% completed

city:    Buraydah
country: SA
	<Response [200]> 323 of 2000 | 16.15% completed

city:    Verkhoyansk
country: RU
	<Response [200]> 324 of 2000 | 16.20% completed

city:    Beringovskiy
country: RU
	<Re

city:    Vaini
country: TO
	<Response [200]> 416 of 2000 | 20.80% completed

city:    Bredasdorp
country: ZA
	<Response [200]> 417 of 2000 | 20.85% completed

city:    Chokurdakh
country: RU
	<Response [200]> 418 of 2000 | 20.90% completed

city:    Arraial Do Cabo
country: BR
	<Response [200]> 419 of 2000 | 20.95% completed

city:    East London
country: ZA
	<Response [200]> 420 of 2000 | 21.00% completed

city:    Khatanga
country: RU
	<Response [200]> 421 of 2000 | 21.05% completed

city:    Kavieng
country: PG
	<Response [200]> 422 of 2000 | 21.10% completed

city:    Klaksvik
country: FO
	<Response [200]> 423 of 2000 | 21.15% completed

city:    Rikitea
country: PF
	<Response [200]> 424 of 2000 | 21.20% completed

city:    Omsukchan
country: RU
	<Response [200]> 425 of 2000 | 21.25% completed

city:    Port Blair
country: IN
	<Response [200]> 426 of 2000 | 21.30% completed

city:    Lebu
country: CL
	<Response [200]> 427 of 2000 | 21.35% completed

city:    Talnakh
country: RU
	<R

...

city:    Churapcha
country: RU
	<Response [200]> 726 of 2000 | 36.30% completed

city:    Kodiak
country: US
	<Response [200]> 727 of 2000 | 36.35% completed

city:    Ushuaia
country: AR
	<Response [200]> 728 of 2000 | 36.40% completed

city:    Aklavik
country: CA
	<Response [200]> 729 of 2000 | 36.45% completed

city:    Bayan
country: KW
	<Response [200]> 730 of 2000 | 36.50% completed

city:    Avarua
country: CK
	<Response [200]> 731 of 2000 | 36.55% completed

city:    Saint George
country: BM
	<Response [200]> 732 of 2000 | 36.60% completed

city:    Busselton
country: AU
	<Response [200]> 733 of 2000 | 36.65% completed

city:    Vila
country: VU
	<Response [200]> 734 of 2000 | 36.70% completed

city:    Bambous Virieux
country: MU
	<Response [200]> 735 of 2000 | 36.75% completed

city:    Tura
country: RU
	<Response [200]> 736 of 2000 | 36.80% completed

city:    Rikitea
country: PF
	<Response [200]> 737 of 2000 | 36.85% completed



Weather data for City Meyungs in Country PW

city:    Port Keats
country: AU
	<Response [200]> 830 of 2000 | 41.50% completed

city:    Kalakamati
country: BW
	<Response [200]> 831 of 2000 | 41.55% completed

city:    Wajir
country: KE
	<Response [200]> 832 of 2000 | 41.60% completed

city:    Albany
country: AU
	<Response [200]> 833 of 2000 | 41.65% completed

city:    Busselton
country: AU
	<Response [200]> 834 of 2000 | 41.70% completed

city:    Sao Jose Da Coroa Grande
country: BR
	<Response [200]> 835 of 2000 | 41.75% completed

city:    Ilulissat
country: GL
	<Response [200]> 836 of 2000 | 41.80% completed

city:    San Patricio
country: MX
	<Response [200]> 837 of 2000 | 41.85% completed

city:    Barrow
country: US
	<Response [200]> 838 of 2000 | 41.90% completed

city:    Victoria
country: SC
	<Response [200]> 839 of 2000 | 41.95% completed

city:    Port Elizabeth
country: ZA
	<Response [200]> 840 of 2000 | 42.00% completed

city:    Cherskiy
country: RU
	<Response [200]> 841 of 2000 | 42.05% completed

city:    Tevait

city:    Kristiinankaupunki
country: FI
	<Response [200]> 933 of 2000 | 46.65% completed

city:    Kavieng
country: PG
	<Response [200]> 934 of 2000 | 46.70% completed

city:    Mataura
country: PF
	<Response [200]> 935 of 2000 | 46.75% completed

city:    Ushuaia
country: AR
	<Response [200]> 936 of 2000 | 46.80% completed

city:    Jamestown
country: SH
	<Response [200]> 937 of 2000 | 46.85% completed

city:    Hilo
country: US
	<Response [200]> 938 of 2000 | 46.90% completed

city:    Butaritari
country: KI
	<Response [200]> 939 of 2000 | 46.95% completed

city:    Hermanus
country: ZA
	<Response [200]> 940 of 2000 | 47.00% completed

city:    Kruisfontein
country: ZA
	<Response [200]> 941 of 2000 | 47.05% completed

city:    Kodiak
country: US
	<Response [200]> 942 of 2000 | 47.10% completed

city:    Punta Arenas
country: CL
	<Response [200]> 943 of 2000 | 47.15% completed

city:    Kamyshevatskaya
country: RU
	<Response [200]> 944 of 2000 | 47.20% completed

city:    Jamestown
co

city:    Ribeira Grande
country: PT
	<Response [200]> 1035 of 2000 | 51.75% completed

city:    Carauari
country: BR
	<Response [200]> 1036 of 2000 | 51.80% completed

city:    Hobart
country: AU
	<Response [200]> 1037 of 2000 | 51.85% completed

city:    Vaini
country: TO
	<Response [200]> 1038 of 2000 | 51.90% completed

city:    Baleshwar
country: IN
	<Response [200]> 1039 of 2000 | 51.95% completed

city:    Sistranda
country: NO
	<Response [200]> 1040 of 2000 | 52.00% completed

city:    Busselton
country: AU
	<Response [200]> 1041 of 2000 | 52.05% completed

city:    Blackwater
country: AU
	<Response [200]> 1042 of 2000 | 52.10% completed

city:    Smithers
country: CA
	<Response [200]> 1043 of 2000 | 52.15% completed

city:    Hermanus
country: ZA
	<Response [200]> 1044 of 2000 | 52.20% completed

city:    Tuktoyaktuk
country: CA
	<Response [200]> 1045 of 2000 | 52.25% completed

city:    Aguimes
country: ES
	<Response [200]> 1046 of 2000 | 52.30% completed

city:    Bathsheba
c

city:    Seoul
country: KR
	<Response [200]> 1137 of 2000 | 56.85% completed

city:    Hermanus
country: ZA
	<Response [200]> 1138 of 2000 | 56.90% completed

city:    Korla
country: CN
	<Response [200]> 1139 of 2000 | 56.95% completed

city:    El Rosario
country: SV
	<Response [200]> 1140 of 2000 | 57.00% completed

city:    Ribeira Grande
country: PT
	<Response [200]> 1141 of 2000 | 57.05% completed

city:    Hervey Bay
country: AU
	<Response [200]> 1142 of 2000 | 57.10% completed

city:    Carutapera
country: BR
	<Response [200]> 1143 of 2000 | 57.15% completed



Weather data for City Attawapiskat in Country CA not found. Skipping...


city:    Jamestown
country: SH
	<Response [200]> 1145 of 2000 | 57.25% completed

city:    Suruc
country: TR
	<Response [200]> 1146 of 2000 | 57.30% completed

city:    Yellowknife
country: CA
	<Response [200]> 1147 of 2000 | 57.35% completed

city:    East London
country: ZA
	<Response [200]> 1148 of 2000 | 57.40% completed

city:    Rikitea
countr

In [ ]:
#create a pandas DataFrame from the list of dictionarys/ array of json objects
world_temp_data_by_city = pd.DataFrame(world_data)
#display part of collected data in the DataFrame
world_temp_data_by_city.head()

In [ ]:
#export Data to a csv
world_temp_data_by_city.to_csv("WeatherPy_Database.csv")
